In [1]:
!pip install opencv-python numpy pytesseract transformers torch torchvision torchaudio ffmpeg moviepy
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install textstat


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6138 sha256=6f53096efd681673a3b7d662ad6dede4e9da241bfa01a2d7fdfeb257ddce1e35
  Stored in directory: c:\users\91630\appdata\local\pip\cache\wheels\26\21\0c\c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built ffmpeg



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/939.4 kB ? eta -:--:--
   --------------------------------------- 939.4/939.4 kB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 23.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install spacy textstat pytesseract pillow
!python -m spacy download en_core_web_sm



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------ --------------------------- 3.9/12.8 MB 21.3 MB/s eta 0:00:01
     -------------- ------------------------- 4.7/12.8 MB 13.0 MB/s eta 0:00:01
     ---------------- ----------------------- 5.2/12.8 MB 11.0 MB/s eta 0:00:01
     ---------------- ----------------------- 5.2/12.8 MB 11.0 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 5.9 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 4.9 MB/s eta 0:00:02
     ------------------------- -------------- 8.1/12.8 MB 5.4 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 5.7 MB/s eta 0:00:01
     -------------------------------- ------- 10.5/12.8 MB 5.6 MB/s eta 0:00:01
     ------------------------------------ --- 11.5/12.8 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 12.8/1


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import os

video_path = "./video_test.mp4"  # Change the path if needed
output_frames_dir = "./frames"

os.makedirs(output_frames_dir, exist_ok=True)
cap = cv2.VideoCapture(video_path)

frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
interval = int(frame_rate * 1)  # Capture every 1 second (adjust as needed)

count = 0
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % interval == 0:  # Capture every Nth frame
        frame_path = f"{output_frames_dir}/frame_{count}.jpg"
        cv2.imwrite(frame_path, frame)
        count += 1
    frame_count += 1

cap.release()
print(f"Extracted {count} frames at {interval} frame intervals.")



Extracted 596 frames at 25 frame intervals.


In [17]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"



In [16]:
!tesseract -v

'tesseract' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
import os
import pytesseract
import textstat  # For readability scoring
import spacy  # For NLP-based topic extraction
from PIL import Image
from collections import Counter

# Load NLP model
nlp = spacy.load("en_core_web_sm")  # Use "en_core_web_md" for better accuracy

def extract_text_from_image(image_path):
    """Extract text from an image using OCR."""
    img = Image.open(image_path)
    img = img.resize((640, 480))  # Resize for optimization
    text = pytesseract.image_to_string(img)
    return text.strip()

# Extract text from limited frames
output_folder = "./frames"  # Ensure this is the correct folder
frame_files = sorted(os.listdir(output_folder))[:20]  # Pick first 20 frames

full_text = ""

for frame in frame_files:
    frame_path = os.path.join(output_folder, frame)
    text = extract_text_from_image(frame_path)
    full_text += " " + text  # Merge all frame texts

# Determine overall difficulty
def get_difficulty(text):
    """Assign difficulty based on text complexity."""
    readability_score = textstat.flesch_kincaid_grade(text)  # Grade level of the text
    if readability_score < 6:
        return "Easy"
    elif readability_score < 10:
        return "Medium"
    else:
        return "Hard"

difficulty_level = get_difficulty(full_text)

# Extract main topic using NLP
def extract_main_topic(text):
    """Extract keywords to determine the main topic."""
    doc = nlp(text)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    most_common = Counter(keywords).most_common(5)  # Get top 5 keywords
    topic = ", ".join([word for word, freq in most_common])
    return topic if topic else "Unknown"

main_topic = extract_main_topic(full_text)

# Print results
print(f"**Overall Difficulty:** {difficulty_level}\n")
print(f"**Main Topic(s):** {main_topic}\n")
print(f"**Extracted Text Preview:**\n{full_text[:1000]}...")  # Print first 1000 characters


TesseractNotFoundError: C:\Program Files\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.